In [1]:
#TDA API testing

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime

In [3]:
import env

credentials loaded successfully


In [4]:
import tda

In [5]:
tda_key = env.tda_api_key
tda_url = env.my_redirect_url

# tda_url

In [6]:
tda

<module 'tda' from '/Users/luke/opt/anaconda3/lib/python3.7/site-packages/tda/__init__.py'>

In [7]:
import os

In [12]:
import atexit
import datetime
import dateutil
import httpx
import sys
import tda

API_KEY = env.tda_api_key
REDIRECT_URI = env.my_redirect_url
TOKEN_PATH = 'ameritrade-token.json'
YOUR_BIRTHDAY = datetime.datetime(year=1986, month=2, day=23)
SP500_URL = "https://tda-api.readthedocs.io/en/latest/_static/sp500.txt"

def make_webdriver():
# Import selenium here because it's slow to import from selenium import webdriver
    from selenium import webdriver
    
    driver = webdriver.Chrome()
    atexit.register(lambda: driver.quit())
    return driver

# Create a new client
client = tda.auth.easy_client(
    API_KEY,
    REDIRECT_URI,
    TOKEN_PATH,
    make_webdriver)


# Load S&P 500 composition from documentation
sp500 = httpx.get(
    SP500_URL, headers={
        "User-Agent": "Mozilla/5.0"}).read().decode().split()

# Fetch fundamentals for all symbols and filter out the ones with ex-dividend
# dates in the future and dividend payment dates on your birth month. Note we
# perform the fetch in two calls because the API places an upper limit on the
# number of symbols you can fetch at once.
today = datetime.datetime.today()
birth_month_dividends = []
for s in (sp500[:250], sp500[250:]):
    r = client.search_instruments(
        s, tda.client.Client.Instrument.Projection.FUNDAMENTAL)
    assert r.status_code == 200, r.raise_for_status()

    for symbol, f in r.json().items():

        # Parse ex-dividend date
        ex_div_string = f['fundamental']['dividendDate']
        if not ex_div_string.strip():
            continue
        ex_dividend_date = dateutil.parser.parse(ex_div_string)

        # Parse payment date
        pay_date_string = f['fundamental']['dividendPayDate']
        if not pay_date_string.strip():
            continue
        pay_date = dateutil.parser.parse(pay_date_string)

        # Check dates
        if (ex_dividend_date > today
                and pay_date.month == YOUR_BIRTHDAY.month):
            birth_month_dividends.append(symbol)

if not birth_month_dividends:
    print('Sorry, no stocks are paying out in your birth month yet. This is ',
          'most likely because the dividends haven\'t been announced yet. ',
          'Try again closer to your birthday.')
    sys.exit(1)

# Purchase one share of each the stocks that pay in your birthday month.
# account_id = int(input(
#     'Input your TDA account number to place orders (<Ctrl-C> to quit): '))
# for symbol in birth_month_dividends:
#     print('Buying one share of', symbol)

    # Build the order spec and place the order
#     order = tda.orders.equities.equity_buy_market(symbol, 1)

#     r = client.place_order(account_id, order)
#     assert r.ok, r.raise_for_status()

In [13]:
SP500_URL

'https://tda-api.readthedocs.io/en/latest/_static/sp500.txt'

In [ ]:
# from tda.auth import easy_client 
# from tda.client import Client

async def main():
    c = tda.easy_client(
        api_key = env.tda_api_key, 
        redirect_uri = env.my_redirect_url,
        token_path = 'ameritrade-token.json', 
        asyncio=True)
    
    resp = await c.get_price_history('AAPL', 
        period_type=Client.PriceHistory.PeriodType.YEAR, 
        period=Client.PriceHistory.Period.TWENTY_YEARS,
        frequency_type=Client.PriceHistory.FrequencyType.DAILY, 
        frequency=Client.PriceHistory.Frequency.DAILY)

    assert resp.status_code == 200
    history = resp.json()

if __name__ == '__main__': 
    import asyncio
    asyncio.run_until_complete(main())

In [16]:
client.get_quote("AAPL")

<Response [200 ]>

In [ ]:
https://api.tdameritrade.com/v1/marketdata/{"AAPL"}/quotes

In [ ]:
# Start here, try to figure out what the path is, and figure out how to get my token setup using a local .json file or other file to link stuff.